# Podcast Episode Erstellen

### imports

In [1]:
import sys
sys.path.append(".")
from audio_transcription.faster_whisper_word_level import transcribe
from Episodes_Downloader.episodes_downloader import get_names_and_urls_all_episodes, download_and_save_mp3_in_dir
import os
import pandas as pd
from pydub import AudioSegment
from pydub.playback import play
from playsound import playsound
import IPython 
import io
from db_connect import db_get_df,db_save_df
from segment_ranking.rank_segments import get_most_similar_documents_Llama2
from Audio_segmentation.split_audio import produce_snippets
from Audio_segmentation.concat_audio import produce_audio
from tqdm import tqdm

/Users/br/Projects/Bachelorarbeit/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
playsound is relying on a python 2 subprocess. Please use `pip3 install PyObjC` if you want playsound to run more efficiently.


### Episoden URLs laden

In [3]:
titles, audio_urls = get_names_and_urls_all_episodes()
print(titles)

['Frieden - Persönliche Sehnsucht - Politische Utopie', 'Selbstliebe - Wie kann man mit sich befreundet sein', 'Philosophie der Schönheit - Was den Menschen "schön" macht', 'Manufakturen - Schritt in eine neue Arbeitswelt', 'Arbeitszeit - Von der Stechuhr zum Coworking Space ', 'Ennio Morricone - Meister der Filmmusik', 'Earl Hamner, Erfinder der Waltons - Sehnsucht nach dem Landleben', 'Warum treiben Menschen Sport? - Eine Erfolgsgeschichte', 'Ernährungstheorien - Fakten oder Mythen?', 'Die Geschichte der Stadt - Urbanes Leben als Motor der Gesellschaft', 'Das Fenster - Geschichte mit Durchblick', 'Tiefenzeit - Vergangenheit und Zukunft wiederentdecken', 'Gegenwart - Ein rätselhaftes Phänomen', 'Vergebung als Chance - Ein Prozess der inneren Aussöhnung', 'Harze - Das Blut der Bäume', 'Runzelige Rinden und bunte Borken - Die Haut der Bäume', 'Tiere im Winter – Alles Natur', 'Eichhörnchen - Kletternde Kobolde', 'Der Eiffelturm - Ikone der Moderne', 'Beruf: Hausfrau - Die Geschichte eine

In [4]:
print(len(titles))

2251


In [5]:
df = pd.DataFrame({"title": titles, "download_url": audio_urls, "transcript": None})
df.dtypes

title           object
download_url    object
transcript      object
dtype: object

In [6]:
db_save_df(df, "transcripts_all")

### Alle mp3s herunterladen

In [7]:
path = "/Volumes/Samsung_T5/Podcast_Episoden"
os.listdir(path)

[]

In [8]:
df = db_get_df("transcripts_all")

for i, row in tqdm(df.iterrows()):
    title = row["title"].replace("/","")
    if title in os.listdir(path):
        continue
    download_and_save_mp3_in_dir(row["download_url"], path, title)


0it [00:00, ?it/s]

1it [00:03,  3.09s/it]

Downloaded https://media.neuland.br.de/file/1865788/c/feed/frieden-persoenliche-sehnsucht-politische-utopie.mp3


2it [00:04,  1.98s/it]

Downloaded https://media.neuland.br.de/file/1795158/c/feed/selbstliebe-wie-kann-man-mit-sich-befreundet-sein.mp3


...

Downloaded https://media.neuland.br.de/file/34536/c/feed/e-t-a-hoffmann-ein-berufsbild.mp3


2250it [42:32,  1.53it/s]

Downloaded https://media.neuland.br.de/file/34513/c/feed/menschen-mythen-religionen-ueber-das-eigentum.mp3


2251it [42:32,  1.13s/it]

Downloaded https://media.neuland.br.de/file/34261/c/feed/hitler-in-wien-als-hitler-noch-unbekannt-war.mp3


In [ ]:
df.loc[179, "download_url"]

### Transkription auf Word level Ebene

In [ ]:
df = transcribe("/Users/br/Projects/Bachelorarbeit/data/Episode_audio_files/jonathan swift gullivers reisen 2.mp3")
# db_save_df(df,"transcript_word_level")

In [ ]:
db_save_df(df, "transcript_gulliver_word_level")

### Debug db

In [11]:
import sqlite3
import pandas as pd

con = sqlite3.connect("/Volumes/Samsung_T5/Backup/transcripts.sqlite")
df = pd.read_sql_query(f"SELECT * FROM transcript_word_level_2237", con)
con.close()

In [16]:
print(df.dtypes)
print(df["filename"].drop_duplicates(ignore_index=True))

filename     object
index       float64
word         object
start       float64
end         float64
dtype: object
0       baruch-de-spinoza-die-suche-nach-der-wahrheit.mp3
1       muenchens-olympiastadion-spinnennetz-und-sensa...
2       sole-salz-und-holz-die-erste-pipeline-stand-in...
3       quentin-tarantino-von-genie-groesse-und-wahnsi...
4       aussteiger-um-1900-nacktkultur-und-barfussphil...
                              ...                        
1940         superorganismen-im-tierreich-alles-natur.mp3
1941          lichtverschmutzung-toedliche-helligkeit.mp3
1942            ueberfischung-wenn-das-meer-leer-wird.mp3
1943               erich-fried-der-politische-lyriker.mp3
1944         autofiktion-wer-schreibt-hier-eigentlich.mp3
Name: filename, Length: 1945, dtype: object


Die Namen der Filenames ändern

In [15]:
df['filename'] = df['filename'].str.replace('/nfs/scratch/students/neumannvi84434/Podcast_Episoden/', '')

In [17]:
with sqlite3.connect("/Volumes/Samsung_T5/Backup/transcripts.sqlite") as con:
    df.to_sql("transcript_word_level_2237", con, index=False, if_exists='replace')

### Satzbildung mit spacy

In [ ]:
import spacy

In [ ]:
df = db_get_df("transcript_gulliver_word_level")
df.dtypes

Zunächst erstmal mit Whispers Punkten

In [ ]:
print(f'|{df["word"][2]}|')

In [ ]:
sentences = []
one_sentence = []
for word in df["word"]:
    one_sentence.append(word.strip())
    if "." in word:
        sentences.append(" ".join(one_sentence))
        one_sentence = []
print(sentences)

### Demonstration

In [ ]:
df = db_get_df(table="transcripts")

In [ ]:
userInputText = "Auf dem l Feiern und trinken"
userInputTime = 7
best_fitting = get_most_similar_documents_Llama2(userInputText, userInputTime)
print(best_fitting["segment_text"].to_markdown())

produce_snippets()
produce_audio()


In [ ]:
IPython.display.Audio("concatenated_audio.mp3")